In [1]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.abspath("../data_model/"))

In [2]:
import enums as e

### Remote I/O

In [3]:
processed_dir = "../data/processed"
interim_dir = "../data/interim"
external_dir = "../data/external"

survey_data_input_file = os.path.join(processed_dir, "data_model_output.csv")
sdia_arrivals_input_file = os.path.join(external_dir, "sdia", "san-daily-arrivals-sept-and-oct-2024-as-csv.csv")
sdia_departures_input_file = os.path.join(external_dir, "sdia", "san-daily-departures-sept-and-oct-2024-as-csv.csv")

output_file = os.path.join(interim_dir, "compare-flight-time-of-day.csv")

### Data Reads

In [4]:
survey_df = pd.read_csv(survey_data_input_file)
arrivals_df = pd.read_csv(sdia_arrivals_input_file)
departures_df = pd.read_csv(sdia_departures_input_file)

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_36560\1798047874.py:1: DtypeWarning: Columns (1,8,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,40,41,47,52,58,63,67,69,71,73,77,84,86,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,122,136,137,140,141,149,151,157,159,161,173,177,186,187,190,193,197,212,214,215,216,217,218,219,220,221,222,223,246,253,254,258,259,261,262,280,298,300,301,303,319,321,343) have mixed types. Specify dtype option on import or set low_memory=False.
  survey_df = pd.read_csv(survey_data_input_file)


### Reductions

In [5]:
arrivals_df.head()

,Date,Mkt Al,Alliance,Op Al,Orig,Dest,Miles,Flight,Stops,Equip,Seats,Dep Term,Arr Term,Dep Time,Arr Time,Block Mins,Arr Flag,Orig WAC,Dest WAC
0,"Sep 30, 2024",AA,oneworld,AA,CLT,SAN,"2,077",2032,0,321,190,NaN,2,714,902,288,0,36,91
1,"Sep 30, 2024",AA,oneworld,AA,CLT,SAN,"2,077",1954,0,321,190,NaN,2,855,1043,288,0,36,91
2,"Sep 30, 2024",AA,oneworld,AA,CLT,SAN,"2,077",2059,0,321,190,NaN,2,1131,1336,305,0,36,91
3,"Sep 30, 2024",AA,oneworld,AA,CLT,SAN,"2,077",1740,0,321,190,NaN,2,1644,1843,299,0,36,91
4,"Sep 30, 2024",AA,oneworld,AA,DFW,SAN,"1,171",2991,0,32Q,196,NaN,2,720,824,184,0,74,91


In [6]:
bin_edges = []
for hour in range(24):
    bin_edges.append(hour * 100)
    bin_edges.append(hour * 100 + 30)

In [7]:
arrivals_df["binned_arrival"] = pd.cut(arrivals_df["Arr Time"], bins=bin_edges, include_lowest=False)
arrivals_summed_df = arrivals_df.groupby(["binned_arrival", "Date", "Arr Term"], observed=True)["Seats"].sum().reset_index()
arrivals_mean_df = arrivals_summed_df.groupby(["binned_arrival", "Arr Term"], observed=True)["Seats"].mean().reset_index()
arrivals_mean_df.columns = ["time_interval", "terminal", "observed_mean_arrival_seats"]
sum(arrivals_mean_df["observed_mean_arrival_seats"])

44999.276697175286

In [8]:
departures_df.head()

,Date,Mkt Al,Alliance,Op Al,Orig,Dest,Miles,Flight,Stops,Equip,Seats,Dep Term,Arr Term,Dep Time,Arr Time,Block Mins,Arr Flag,Orig WAC,Dest WAC
0,"Nov 1, 2024",AA,oneworld,AA,SAN,CLT,"2,077",598,0,321,190,2,NaN,645,1427,282,0,91,36
1,"Nov 1, 2024",AA,oneworld,AA,SAN,CLT,"2,077",1645,0,321,190,2,NaN,1204,1942,278,0,91,36
2,"Nov 1, 2024",AA,oneworld,AA,SAN,CLT,"2,077",2512,0,321,190,2,NaN,2125,503,278,1,91,36
3,"Nov 1, 2024",AA,oneworld,AA,SAN,CLT,"2,077",2056,0,321,190,2,NaN,2210,548,278,1,91,36
4,"Nov 1, 2024",AA,oneworld,AA,SAN,DFW,"1,171",2090,0,321,190,2,NaN,615,1126,191,0,91,74


In [9]:
departures_df["binned_departure"] = pd.cut(departures_df["Dep Time"], bins=bin_edges, include_lowest=False)
departures_summed_df = departures_df.groupby(["binned_departure", "Date", "Dep Term"], observed=True)["Seats"].sum().reset_index()
departures_mean_df = departures_summed_df.groupby(["binned_departure", "Dep Term"], observed=True)["Seats"].mean().reset_index()
departures_mean_df.columns = ["time_interval", "terminal", "observed_mean_departure_seats"]
sum(departures_mean_df["observed_mean_departure_seats"])

44851.606438016

In [10]:
start_time = 500
interval_duration = 30
intervals = []
for i in range(48):
    start = start_time + i * interval_duration
    end = start + interval_duration
    intervals.append((start, end))

interval_index = pd.IntervalIndex.from_tuples(intervals, closed='right')

time_index_map_df = pd.DataFrame(interval_index, columns=["time_interval"])

# add a column with the index values
time_index_map_df["time_index"] = range(1, 49)
time_index_map_df.head()

,time_interval,time_index
0,"(500, 530]",1
1,"(530, 560]",2
2,"(560, 590]",3
3,"(590, 620]",4
4,"(620, 650]",5


In [11]:
intervals = [
    (500, 530),
    (530, 600),
    (600, 630),
    (630, 700),
    (700, 730),
    (730, 800),
    (800, 830),
    (830, 900),
    (900, 930),
    (930, 1000),
    (1000, 1030),
    (1030, 1100),
    (1100, 1130),
    (1130, 1200),
    (1200, 1230),
    (1230, 1300),
    (1300, 1330),
    (1330, 1400),
    (1400, 1430),
    (1430, 1500),
    (1500, 1530),
    (1530, 1600),
    (1600, 1630),
    (1630, 1700),
    (1700, 1730),
    (1730, 1800),
    (1800, 1830),
    (1830, 1900),
    (1900, 1930),
    (1930, 2000),
    (2000, 2030),
    (2030, 2100),
    (2100, 2130),
    (2130, 2200),
    (2200, 2230),
    (2230, 2300),
    (2300, 2330),
    (2330, 2400),
    (0, 30),
    (30, 100),
    (100, 130),
    (130, 200),
    (200, 230),
    (230, 300),
    (300, 330),
    (330, 400),
    (400, 430),
    (430, 500)
]

interval_index = pd.IntervalIndex.from_tuples(intervals, closed="right")
index_map_df = pd.DataFrame(interval_index, columns=["time_interval"])
index_map_df["time_index"] = range(1, 49)

In [12]:
observed_df = pd.merge(arrivals_mean_df, departures_mean_df, on=["time_interval", "terminal"])
observed_df = pd.merge(observed_df, index_map_df, on=["time_interval"], how="left")
observed_df.head()

,time_interval,terminal,observed_mean_arrival_seats,observed_mean_departure_seats,time_index
0,"(630, 700]",1,232.000000,665.15625,4
1,"(700, 730]",1,634.904762,506.34375,5
2,"(700, 730]",2,377.718750,779.18750,5
3,"(730, 800]",1,747.593750,917.78125,6
4,"(730, 800]",2,381.838710,1762.50000,6


In [13]:
working_df = survey_df[[
    "unique_id", "is_completed", "inbound_or_outbound", "marketsegment", "airport_terminal", "flight_arrival_time", "flight_departure_time", "party_size_flight",  "weight"
    ]].copy()
working_df = working_df.loc[working_df["marketsegment"] == e.Type.PASSENGER].copy()
working_df["travelers"] = working_df["weight"] * working_df["party_size_flight"]
working_df.head()

,unique_id,is_completed,inbound_or_outbound,marketsegment,airport_terminal,flight_arrival_time,flight_departure_time,party_size_flight,weight,travelers
458,459,True,2.0,1,1.0,17.0,NaN,1.0,NaN,NaN
459,460,True,2.0,1,2.0,20.0,NaN,0.0,NaN,NaN
460,461,True,2.0,1,2.0,24.0,NaN,0.0,NaN,NaN
461,462,True,2.0,1,2.0,17.0,NaN,0.0,NaN,NaN
462,463,True,2.0,1,1.0,18.0,NaN,0.0,NaN,NaN


In [14]:
survey_arrivals_df = working_df.groupby(["flight_arrival_time", "airport_terminal"], observed=True)["travelers"].agg(["sum", "count"]).reset_index()
survey_arrivals_df.columns = ["time_index", "terminal", "survey_arrivals", "survey_arrivals_observations"]

survey_departures_df = working_df.groupby(["flight_departure_time", "airport_terminal"], observed=True)["travelers"].agg(["sum", "count"]).reset_index()
survey_departures_df.columns = ["time_index", "terminal", "survey_departures", "survey_departures_observations"]

join_df = pd.merge(survey_arrivals_df, survey_departures_df, on=["time_index", "terminal"])
join_df["terminal"] = join_df["terminal"].astype("int")
join_df["time_index"] = join_df["time_index"].astype("int")
join_df.head()

,time_index,terminal,survey_arrivals,survey_arrivals_observations,survey_departures,survey_departures_observations
0,3,2,0.0,0,10.289756,2
1,4,2,0.0,0,81.204221,4
2,5,1,0.0,0,309.356872,4
3,5,2,0.0,0,25.508248,22
4,6,1,0.0,0,417.178609,2


In [15]:
output_df = pd.merge(observed_df, join_df, on=["time_index", "terminal"], how = "outer")
output_df["time_label"] = output_df["time_index"].apply(lambda x: e.DepartTime(x).name)
output_df["time_interval"] = output_df["time_interval"].fillna(output_df["time_index"].map(index_map_df.set_index("time_index")["time_interval"].to_dict()))
output_df = output_df.fillna(0.0)
output_df.head()

,time_interval,terminal,observed_mean_arrival_seats,observed_mean_departure_seats,time_index,survey_arrivals,survey_arrivals_observations,survey_departures,survey_departures_observations,time_label
0,"(600.0, 630.0]",2,0.000000,0.00000,3,0.0,0.0,10.289756,2.0,SIX_TO_SIX_THIRTY
1,"(630.0, 700.0]",1,232.000000,665.15625,4,0.0,0.0,0.000000,0.0,SIX_THIRTY_TO_SEVEN
2,"(630.0, 700.0]",2,0.000000,0.00000,4,0.0,0.0,81.204221,4.0,SIX_THIRTY_TO_SEVEN
3,"(700.0, 730.0]",1,634.904762,506.34375,5,0.0,0.0,309.356872,4.0,SEVEN_TO_SEVEN_THIRTY
4,"(700.0, 730.0]",2,377.718750,779.18750,5,0.0,0.0,25.508248,22.0,SEVEN_TO_SEVEN_THIRTY


In [16]:
output_df.to_csv(output_file, index=False)